<a href="https://colab.research.google.com/github/gbailey46/public/blob/master/Feline_Surveillance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

Google Drive access.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
!ls -al /root/.keras/datasets/
#!rm /root/.keras/datasets/feline_surveillance*
!unzip "/content/drive/My Drive/ML_Datasets/feline_surveillance.zip" -d "/root/.keras/datasets/"

In [0]:
PATH = "/root/.keras/datasets/feline_surveillance"

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_belle_dir = os.path.join(train_dir, 'belle')  # directory with our training belle pictures
train_millie_dir = os.path.join(train_dir, 'millie')  # directory with our training millie pictures
train_lara_dir = os.path.join(train_dir, 'lara')  # directory with our training lara pictures
validation_belle_dir = os.path.join(validation_dir, 'belle')  # directory with our validation belle pictures
validation_millie_dir = os.path.join(validation_dir, 'millie')  # directory with our validation millie pictures
validation_lara_dir = os.path.join(validation_dir, 'lara')  # directory with our validation lara pictures

In [0]:
num_belle_tr = len(os.listdir(train_belle_dir))
num_millie_tr = len(os.listdir(train_millie_dir))
num_lara_tr = len(os.listdir(train_lara_dir))

num_belle_val = len(os.listdir(validation_belle_dir))
num_millie_val = len(os.listdir(validation_millie_dir))
num_lara_val = len(os.listdir(validation_lara_dir))

total_train = num_belle_tr + num_millie_tr + num_lara_tr
total_val = num_belle_val + num_millie_val + num_lara_val

In [0]:
print('total training belle images:', num_belle_tr)
print('total training millie images:', num_millie_tr)
print('total training lara images:', num_lara_tr)

print('total validation belle images:', num_belle_val)
print('total validation millie images:', num_millie_val)
print('total validation lara images:', num_lara_val)

print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

In [0]:
batch_size = 25
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [0]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [0]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [0]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

In [0]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [0]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_new.summary()

In [0]:
history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()